# 1. Import Data

In [1]:
import pandas as pd
import numpy as np
import random
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_colwidth', 150000) #important for getting all the text
pd.set_option('display.max_columns', 999)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from IPython.display import Image  
import random

import scipy
import math
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [2]:
#Load the data
ratings_df = pd.read_csv('ratings.csv')
movies_df = pd.read_csv('movies.csv')

ratings_df.head()
movies_df.head()

,Unnamed: 0,movieId,title,directors,writers,stars,year,certificate,length (min),Storyline,languages,Oscars,genres,popularity,Rating
0,0,5,Father of the Bride Part II,['charles_shyer'],"['albert_hackett', 'frances_goodrich', 'nancy_meyers']","['steve_martin', 'diane_keaton', 'martin_short']",1995,PG,106,"George Banks must deal not only with his daughter's pregnancy, but also with his wife's.",['English'],NaN,['Comedy'],popular,Low_rated
1,1,1,Toy Story,['john_lasseter'],"['john_lasseter', 'pete_docter', 'andrew_stanton']","['tom_hanks', 'tim_allen', 'don_rickles']",1995,G,81,A cowboy doll is profoundly threatened and jealous when a new spaceman action figure supplants him as top toy in a boy's bedroom.,"['English ', ' Russian ', ' Ukrainian']",Nominated,"['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']",popular,High_rated
2,2,6,Heat,['michael_mann'],['michael_mann'],"['al_pacino', 'robert_de_niro', 'val_kilmer']",1995,R,170,A group of high-end professional thieves start to feel the heat from the LAPD when they unknowingly leave a verbal clue at their latest heist.,"['English ', ' Spanish']",NaN,"['Action', 'Crime', 'Thriller']",popular,High_rated
3,3,3,Grumpier Old Men,['howard_deutch'],['mark_steven_johnson'],"['walter_matthau', 'jack_lemmon', 'ann-margret']",1995,PG-13,101,"John and Max resolve to save their beloved bait shop from turning into an Italian restaurant, just as its new female owner catches Max's attention.","['English ', ' Italian ', ' German']",NaN,"['Comedy', 'Romance']",popular,Low_rated
4,4,7,Sabrina,['sydney_pollack'],"['samuel_a._taylor', 'billy_wilder', 'ernest_lehman']","['harrison_ford', 'julia_ormond', 'greg_kinnear']",1995,PG,127,"An ugly duckling having undergone a remarkable change, still harbors feelings for her crush: a carefree playboy, but not before his business-focused brother has something to say about it.","['English ', ' French']",Nominated,"['Comedy', 'Romance']",popular,Low_rated


In [3]:
ratings_with_titles_df = ratings_df.merge(movies_df, on='movieId', how='left')

ratings_with_titles_df.head()

,userId,movieId,rating,timestamp,Unnamed: 0,title,directors,writers,stars,year,certificate,length (min),Storyline,languages,Oscars,genres,popularity,Rating
0,1,1,4.0,964982703,1.0,Toy Story,['john_lasseter'],"['john_lasseter', 'pete_docter', 'andrew_stanton']","['tom_hanks', 'tim_allen', 'don_rickles']",1995,G,81.0,A cowboy doll is profoundly threatened and jealous when a new spaceman action figure supplants him as top toy in a boy's bedroom.,"['English ', ' Russian ', ' Ukrainian']",Nominated,"['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']",popular,High_rated
1,1,3,4.0,964981247,3.0,Grumpier Old Men,['howard_deutch'],['mark_steven_johnson'],"['walter_matthau', 'jack_lemmon', 'ann-margret']",1995,PG-13,101.0,"John and Max resolve to save their beloved bait shop from turning into an Italian restaurant, just as its new female owner catches Max's attention.","['English ', ' Italian ', ' German']",NaN,"['Comedy', 'Romance']",popular,Low_rated
2,1,6,4.0,964982224,2.0,Heat,['michael_mann'],['michael_mann'],"['al_pacino', 'robert_de_niro', 'val_kilmer']",1995,R,170.0,A group of high-end professional thieves start to feel the heat from the LAPD when they unknowingly leave a verbal clue at their latest heist.,"['English ', ' Spanish']",NaN,"['Action', 'Crime', 'Thriller']",popular,High_rated
3,1,47,5.0,964983815,39.0,Seven (a.k.a. Se7en),['david_fincher'],['andrew_kevin_walker'],"['morgan_freeman', 'brad_pitt', 'kevin_spacey']",1995,R,127.0,"Two detectives, a rookie and a veteran, hunt a serial killer who uses the seven deadly sins as his motives.",['English'],Nominated,"['Mystery', 'Thriller']",popular,High_rated
4,1,50,5.0,964982931,44.0,The Usual Suspects,['bryan_singer'],['christopher_mcquarrie'],"['kevin_spacey', 'gabriel_byrne', 'chazz_palminteri']",1995,R,106.0,The sole survivor of a pier shoot-out tells the story of how a notorious criminal influenced the events that began with five criminals meeting in a seemingly random police lineup.,"['English ', ' Hungarian ', ' Spanish ', ' French']",Won,"['Crime', 'Mystery', 'Thriller']",popular,High_rated


# 2. Evaluation 

## 2.1 Data Split

In [4]:
# split dataset with test size = 0.2
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(ratings_df,
stratify=ratings_df['userId'],
test_size=0.20,
random_state=42)

train_df.shape, test_df.shape

((80668, 4), (20168, 4))

In [5]:
# Indexing by 'UserID' to speed up the searches during evaluation
full_indexed_df = ratings_df.set_index('userId')
train_indexed_df = train_df.set_index('userId')
test_indexed_df = test_df.set_index('userId')
full_indexed_df, train_indexed_df

(        movieId  rating   timestamp
 userId                             
 1             1     4.0   964982703
 1             3     4.0   964981247
 1             6     4.0   964982224
 1            47     5.0   964983815
 1            50     5.0   964982931
 ...         ...     ...         ...
 610      166534     4.0  1493848402
 610      168248     5.0  1493850091
 610      168250     5.0  1494273047
 610      168252     5.0  1493846352
 610      170875     3.0  1493846415
 
 [100836 rows x 3 columns],
         movieId  rating   timestamp
 userId                             
 140        4234     3.0  1012505945
 156        2080     1.0   951113118
 380      182639     4.0  1536874706
 495        5254     3.5  1458636268
 610       69134     3.0  1493848172
 ...         ...     ...         ...
 318      136024     3.0  1455886225
 432        5507     1.5  1315242710
 474        4012     2.0  1046887657
 510         497     1.5  1141158809
 274       51709     3.5  1285897528
 
 [8066

## 2.2 Evaluation Class

In [6]:
# Correcting the oversight by adding the evaluate_model method to the ModelEvaluator class
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100
class ModelEvaluator:

    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, full_indexed_df)
        all_items = list(set(ratings_with_titles_df['movieId']))  # Convert to a list
        non_interacted_items = list(set(all_items) - set(interacted_items))  # Convert to a list

        random.seed(seed)

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            non_interacted_items_sample = random.sample(non_interacted_items, sample_size)

        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):
        try:
            index = next(i for i, c in enumerate(recommended_items) if c == item_id)
        except:
            index = -1
        hit = int(index in range(0, topn))
        return hit, index

    def evaluate_model_for_user(self, model, person_id):
        interacted_values_testset = test_indexed_df.loc[person_id]
        if type(interacted_values_testset['movieId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['movieId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['movieId'])])
        interacted_items_count_testset = len(person_interacted_items_testset)

        person_recs_df = model.recommend_items(person_id,
                                               items_to_ignore=get_items_interacted(person_id, train_indexed_df),
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0

        for item_id in person_interacted_items_testset:
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id,
                                                                               sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS,
                                                                               seed=item_id % (2**32))
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            valid_recs_df = person_recs_df[person_recs_df['movieId'].isin(items_to_filter_recs)]
            valid_recs = valid_recs_df['movieId'].values

            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count': hits_at_5_count,
                          'hits@10_count': hits_at_10_count,
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        people_metrics = []
        for idx, person_id in enumerate(list(test_indexed_df.index.unique().values)):
            person_metrics = self.evaluate_model_for_user(model, person_id)
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        
        detailed_results_df = pd.DataFrame(people_metrics).sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}
        
        return global_metrics, detailed_results_df

## 2.3 Get Items Interacted Function

In [7]:
# Get all movieIDs that the given user has interacted with
def get_items_interacted(person_id, ratings_df):
    interacted_items = ratings_df.loc[person_id]['movieId'] # interacted (movies') ids
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

# 3.Baseline Model

## 3.1 weighted rating

$$
WR = \left( \frac{v}{v+m} \cdot R \right) + \left( \frac{m}{v+m} \cdot C \right)
$$
Where,
$$
\begin{align*}
v & \text{ is the number of votes for the movie,} \\
m & \text{ is the minimum votes required to be listed in the chart.}\\
R & \text{ is the average rating of the movie}\\
C & \text{ is the mean vote across the dataset}
\end{align*}
$$

To determine an appropriate value for m, the minimum votes required to be listed in the chart. We will use 95th percentile as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 95% of the movies in the list.

In [8]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
# Calculate averageRating for each movie
average_ratings_new = ratings_df.groupby('movieId')['rating'].mean().reset_index(name='averageRating')

# Calculate ratingCount for each movie
rating_counts_new = ratings_df.groupby('movieId').size().reset_index(name='ratingCount')

#Creating a new DataFrame that includes movieId, averageRating, and ratingCount
movie_details_df = pd.merge(average_ratings_new, rating_counts_new, on='movieId')
movie_details_df.head()

,movieId,averageRating,ratingCount
0,1,3.920930,215
1,2,3.431818,110
2,3,3.259615,52
3,4,2.357143,7
4,5,3.071429,49


In [10]:
# Calculate the mean vote C across the whole report
C= movie_details_df['averageRating'].mean()
C

3.262448274810963

In [11]:
#Calculate the minimum votes required to be listed in the chart
m = movie_details_df['ratingCount'].quantile(0.95)
m

47.0

In [12]:
q_movies =movie_details_df.copy().loc[movie_details_df['ratingCount'] >= m]
q_movies.shape
q_movies

,movieId,averageRating,ratingCount
0,1,3.920930,215
1,2,3.431818,110
2,3,3.259615,52
4,5,3.071429,49
5,6,3.946078,102
...,...,...,...
8457,112852,4.050847,59
8551,116797,4.020000,50
8663,122882,3.819149,47
8673,122904,3.833333,54


In [13]:
def weighted_rating(x, m=m, C=C):
    v = x['ratingCount']
    R = x['averageRating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [14]:
q_movies['rating'] = q_movies.apply(weighted_rating, axis=1)

In [15]:
q_movies_with_titles = pd.merge(q_movies, movies_df[['movieId', 'title']], on='movieId', how='left')

In [16]:
q_movies_with_titles = q_movies_with_titles.sort_values('rating', ascending=False)
q_movies_with_titles

,movieId,averageRating,ratingCount,rating,title
56,318,4.429022,317,4.278393,The Shawshank Redemption
301,2959,4.272936,218,4.093717,Fight Club
124,858,4.289062,192,4.087176,The Godfather
47,260,4.231076,251,4.078306,Star Wars: Episode IV - A New Hope
52,296,4.197068,307,4.072980,Pulp Fiction
...,...,...,...,...,...
123,788,2.731707,82,2.925078,The Nutty Professor
324,3623,2.714286,77,2.922057,Mission: Impossible II
10,19,2.727273,88,2.913593,Ace Ventura: When Nature Calls
77,435,2.420635,63,2.780319,Coneheads


## 3.2 Weighted Rating Recommendation Model

In [17]:
class WeightedRatingRecommender:
    
    MODEL_NAME = 'WeightedRating'
    
    def __init__(self, wr_df, items_df=None):
        self.wr_df = wr_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the item higher weighted rating that the user hasn't seen yet.
        recommendations_df = self.wr_df[~self.wr_df['movieId'].isin(items_to_ignore)] \
                               .sort_values('rating', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            
            recommendations_df = recommendations_df.merge(self.items_df, how='left', 
                                                          left_on='movieId', 
                                                          right_on='movieId')[['rating', 'movieId', 'title']]
        return recommendations_df

In [18]:
weightRating_model = WeightedRatingRecommender(q_movies,movies_df)

In [19]:
model_evaluator_wr= ModelEvaluator()

### 3.2.1 Evaluation Result (Baseline)

In [20]:
print('Evaluating Weighted Rating recommendation model...')
wr_global_metrics, wr_detailed_results_df = model_evaluator_wr.evaluate_model(weightRating_model)

print('\nGlobal metrics:\n%s' % wr_global_metrics)
wr_detailed_results_df.head(10)

Evaluating Weighted Rating recommendation model...

Global metrics:
{'modelName': 'WeightedRating', 'recall@5': 0.40202300674335584, 'recall@10': 0.43018643395477985}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
4,83,83,540,0.153704,0.153704,414
29,81,81,496,0.163306,0.163306,599
86,62,62,422,0.146919,0.146919,474
40,60,62,373,0.160858,0.166220,448
176,77,77,269,0.286245,0.286245,274
32,44,45,261,0.168582,0.172414,610
51,89,89,252,0.353175,0.353175,68
133,55,56,244,0.225410,0.229508,380
147,48,49,223,0.215247,0.219731,606
1,72,73,211,0.341232,0.345972,288


### 3.2.2 Recommendation List for user_id = 1 (Baseline)

In [21]:
user_id = 1

# Get recommendations for the specified user
recommendations = weightRating_model.recommend_items(user_id,verbose=True)
print(recommendations)

     rating  movieId                                            title
0  4.278393      318                         The Shawshank Redemption
1  4.093717     2959                                      Fight Club 
2  4.087176      858                                    The Godfather
3  4.078306      260              Star Wars: Episode IV - A New Hope 
4  4.072980      296                                    Pulp Fiction 
5  4.057954     2571                                       The Matrix
6  4.055562      527                                Schindler's List 
7  4.055120       50                               The Usual Suspects
8  4.051423      356                                    Forrest Gump 
9  4.041996     1196  Star Wars: Episode V - The Empire Strikes Back 


# 4. Content-Based Filtering Model

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
import math

## 4.1 Assign Weights to Features

In [23]:
def adjust_feature_frequency(row, feature_weights):
    adjusted_text = ""
    for feature, weight in feature_weights.items():
        feature_value = str(row[feature])  # convert to str
        repeated_feature = ' '.join([feature_value] * weight)  # Repeat and join with spaces
        adjusted_text += " " + repeated_feature  # Append with space for separation
    return adjusted_text.strip()  # Remove leading/trailing spaces

In [24]:
# Define the weights
feature_weights = {
    'title': 1,  # Base weight, repeated once
    'directors': 2,  # Twice as important as the title
    'writers': 1,
    'stars': 2,
    'year': 4,
    'certificate': 1,
    'length (min)': 1,
    'Storyline': 1,
    'languages': 1,
    'Oscars': 3,  # thrice as important as the title
    'genres': 2,
    'popularity': 1,
    'Rating': 1,
}

In [25]:
# Adjust feature frequencies based on weights and combine into one column
movies_df['combined'] = movies_df.apply(lambda row: adjust_feature_frequency(row, feature_weights), axis=1)

## 4.2 Create Matrix & Get User Profile

In [26]:
# Return the list of movie_is that a specific user rates
# def get_items_interacted(user_id, movies_df):
#    # Get the user's data and merge in the movie information.
#    interacted_items = movies_df.loc[user_id]['movieId']
#    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [27]:
tfidf1 = TfidfVectorizer(lowercase=True, 
                        stop_words= 'english',
                        max_df=0.95,
                        min_df=0.003,
                        ngram_range = (1,2)) 

In [28]:
tfidf_matrix = tfidf1.fit_transform(movies_df.combined)
print(tfidf_matrix.shape)

(9739, 1619)


In [29]:
tfidf1_feature=tfidf1.get_feature_names_out()
count = np.sum(tfidf_matrix.toarray(), axis = 0).tolist()
count_tfidf = pd.DataFrame(count, index = tfidf1_feature, columns = ['count'])
count_tfidf.sort_values(['count'], ascending = False).head(20)

,count
nan,840.146462
nan nan,551.265747
drama,454.162297
comedy,429.717838
nominated,298.073571
english,278.657220
thriller,271.094513
romance,260.034010
action,259.410070
unpopular,242.786742


In [30]:
from sklearn.feature_extraction import text 
skl_stopwords = list(text.ENGLISH_STOP_WORDS)
my_stopwords = skl_stopwords + ['nan']

In [31]:
tfidf1 = TfidfVectorizer(lowercase=True, 
                        stop_words= my_stopwords, 
                        max_df=0.95,
                        min_df=0.003,
                        ngram_range = (1,2)) 

In [32]:
tfidf_matrix = tfidf1.fit_transform(movies_df.combined)
print(tfidf_matrix.shape)

(9739, 1623)


In [33]:
item_ids = movies_df['movieId'].tolist()

tfidf_feature_names = tfidf1.get_feature_names_out()
count = np.sum(tfidf_matrix.toarray(), axis = 0).tolist()
count_tfidf = pd.DataFrame(count, index = tfidf_feature_names, columns = ['count'])
count_tfidf.sort_values(['count'], ascending = False).head(20)

,count
drama,457.531719
comedy,433.587928
nominated,298.079067
english,280.989389
thriller,273.406442
romance,262.078693
action,261.018905
unpopular,245.235460
high_rated,243.609304
popular,231.519227


In [34]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['movieId'])
    
    user_item_strengths = np.array(interactions_person_df['rating']).reshape(-1,1)
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = sklearn.preprocessing.normalize(np.asarray(user_item_strengths_weighted_avg))
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = train_df[train_df['movieId'] \
                                                   .isin(movies_df['movieId'])].set_index('userId')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [35]:
user_profiles = build_users_profiles()
len(user_profiles)

610

## 4.3 CB Recommendation Model with cosine similarity

In [36]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['movieId', 'recRating']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'movieId', 
                                                          right_on = 'movieId')[['recRating', 'movieId', 'title']]


        return recommendations_df

In [37]:
content_based_recommender_model = ContentBasedRecommender(movies_df)  

In [38]:
model_evaluator_cb= ModelEvaluator()

### 4.3.1 Evaluation Result (CB)

In [39]:
print('Evaluating Conetnt_based recommendation model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator_cb.evaluate_model(content_based_recommender_model)

print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Conetnt_based recommendation model...

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.3495140817136057, 'recall@10': 0.46360571201904005}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
4,85,108,540,0.157407,0.200000,414
29,71,95,496,0.143145,0.191532,599
86,117,132,422,0.277251,0.312796,474
40,66,80,373,0.176944,0.214477,448
176,71,88,269,0.263941,0.327138,274
32,81,93,261,0.310345,0.356322,610
51,58,71,252,0.230159,0.281746,68
133,65,82,244,0.266393,0.336066,380
147,50,70,223,0.224215,0.313901,606
1,65,79,211,0.308057,0.374408,288


### 4.3.2 Recommendation List for user_id = 1 (CB)

In [40]:
user_id = 1

# Get recommendations for the specified user
recommendations_cb = content_based_recommender_model.recommend_items(user_id,verbose=True)

print(recommendations_cb)

   recRating  movieId                           title
0   0.428843     3114                    Toy Story 2 
1   0.425772     2355                  Bug's Life, A 
2   0.422217     2617                       The Mummy
3   0.405397        1                      Toy Story 
4   0.403256     2429               Mighty Joe Young 
5   0.402841     2687                         Tarzan 
6   0.397531     1881              Quest for Camelot 
7   0.394680     1356       Star Trek: First Contact 
8   0.393017      780  Independence Day (a.k.a. ID4) 
9   0.392767     1907                          Mulan 


## 4.4 Cross Validation (CB)

In [41]:
import pandas as pd
from sklearn.model_selection import KFold


# Assuming you have interactions_indexed_df containing user-item interactions
# Define your interactions_test_indexed_df and interactions_train_indexed_df accordingly

# Initialize ModelEvaluator
model_evaluator = ModelEvaluator()

# Define the number of folds
n_splits = 5

# Initialize KFold cross-validator
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store evaluation results for each fold
global_metrics_list = []
detailed_results_df_list = []

# Perform cross-validation
for fold_idx, (train_index, test_index) in enumerate(kf.split(full_indexed_df)):
    print(f"Fold {fold_idx + 1}/{n_splits}")
    
    # Split data into training and test sets for the current fold
    interactions_train_fold = full_indexed_df.iloc[train_index]
    interactions_test_fold = full_indexed_df.iloc[test_index]

    # Convert to interaction train and test sets (assuming you have defined these)
    train_indexed_df = interactions_train_fold  # Define your interaction train set
    test_indexed_df = interactions_test_fold  # Define your interaction test set

    # Evaluate model for the current fold
    global_metrics_fold, detailed_results_df_fold = model_evaluator.evaluate_model(content_based_recommender_model)
    print('\nGlobal metrics:\n%s' % global_metrics_fold)
    # Append results to lists
    global_metrics_list.append(global_metrics_fold)
    detailed_results_df_list.append(detailed_results_df_fold)

Fold 1/5

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.42205474018246725, 'recall@10': 0.5295517651725505}
Fold 2/5

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.42083601923935143, 'recall@10': 0.5316110477512769}
Fold 3/5

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.42529875539247286, 'recall@10': 0.5399910745276938}
Fold 4/5

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.4241086924183071, 'recall@10': 0.5364704715624535}
Fold 5/5

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.419993057965984, 'recall@10': 0.5270987256408985}


In [45]:
# Calculate average global metrics across all folds
global_metrics_df = pd.DataFrame(global_metrics_list)
numeric_cols = global_metrics_df.select_dtypes(include=['float64', 'int64']).columns
avg_global_metrics_cb = global_metrics_df[numeric_cols].mean()

# Display average global metrics
print("\nAverage Global Metrics across all folds:")
print(avg_global_metrics_cb)


Average Global Metrics across all folds:
recall@5     0.422458
recall@10    0.532945
dtype: float64


# 5. Collaborative Flitering - SVD

## 5.1 Creat the sparse pivot table

In [46]:
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

# Creating a sparse pivot table with users in rows and items in columns
users_items_pivot_matrix_df = train_df.pivot(index='userId', 
                                             columns='movieId',                     
                                             values='rating').fillna(0)
users_items_pivot_matrix_df.head(10)

movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     0.0     4.0     0.0     0.0     4.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
6           0.0     0.0     5.0     3.0     0.0     4.0     0.0     3.0   
7           4.5     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
8           0.0     4.0     0.0     0.0     0.0     0.0     0.0     0.0   
9           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
10          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  9       10      11      12      13      14      15      16      \
userId                                                                    
1           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
6           0.0     3.0     4.0     0.0     3.0     0.0     4.0     4.0   
7           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
8           0.0     2.0     0.0     0.0     0.0     0.0     0.0     0.0   
9           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
10          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  17      18      19      20      21      22      23      24      \
userId                                                                    
1           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           0.0     0.0     0.0     0.0     4.0     0.0     0.0     0.0   
6           0.0     0.0     2.0     0.0     2.0     5.0     0.0     4.0   
7           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
8           0.0     0.0     0.0     0.0     4.0     0.0     0.0     0.0   
9           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
10          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  25      26      27      28      29      30      31      32      \
userId                                                                    
1           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.5     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     2.0   
5           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
6           3.0     4.0     0.0     0.0     0.0     0.0     3.0     4.0   
7           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
8           0.0     0.0     0.0     0.0     0.0     0.0     0.0     3.0   
9           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
10          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  34      36      38      39      40      41      42      43      \
userId                                                                    
1           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0

In [47]:
users_items_pivot_matrix = users_items_pivot_matrix_df.to_numpy()
users_items_pivot_matrix[:10]

array([[4., 0., 4., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 4., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [48]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [49]:
users_items_pivot_sparse_matrix = csr_matrix(users_items_pivot_matrix)
users_items_pivot_sparse_matrix

<610x8977 sparse matrix of type '<class 'numpy.float64'>'
	with 80668 stored elements in Compressed Sparse Row format>

## 5.2 Apply the matrix factorization (initialize K = 10)

In [50]:
# Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_items_pivot_sparse_matrix, k = 10)
sigma = np.diag(sigma) # diagonize

### 5.2.1 Predict Ratings by SVD

In [51]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[ 2.41788452e+00,  5.03766709e-01,  7.99193198e-01, ...,
        -1.07615673e-02, -1.07615673e-02, -1.03259727e-02],
       [ 1.05526520e-01,  3.66777140e-02, -6.10915243e-02, ...,
         4.40604594e-03,  4.40604594e-03,  8.02074073e-03],
       [ 3.00340525e-02,  1.31511940e-02,  1.97063911e-02, ...,
         1.92107015e-04,  1.92107015e-04, -1.23752636e-03],
       ...,
       [ 3.67407188e+00,  1.72764716e+00,  1.49032712e+00, ...,
        -4.67555049e-02, -4.67555049e-02,  1.27997790e-03],
       [ 6.79041517e-01,  4.96941395e-01,  2.61196334e-01, ...,
        -4.13231037e-04, -4.13231037e-04, -4.91896691e-04],
       [-3.72792300e-01,  1.38841364e+00, -2.82613680e-01, ...,
         3.98974774e-02,  3.98974774e-02,  6.00590680e-02]])

### 5.2.2 Normalize Prediction

In [52]:
all_user_predicted_ratings_norm = (all_user_predicted_ratings - all_user_predicted_ratings.min()) / \
                                    (all_user_predicted_ratings.max() - all_user_predicted_ratings.min())
all_user_predicted_ratings_norm

array([[0.47830581, 0.33357604, 0.35591375, ..., 0.29467166, 0.29467166,
        0.2947046 ],
       [0.30346441, 0.29825863, 0.29086613, ..., 0.29581851, 0.29581851,
        0.29609183],
       [0.29775629, 0.29647975, 0.2969754 , ..., 0.29549989, 0.29549989,
        0.29539179],
       ...,
       [0.57328832, 0.42611577, 0.40817158, ..., 0.2919501 , 0.2919501 ,
        0.29558215],
       [0.34682887, 0.33305997, 0.31523487, ..., 0.29545412, 0.29545412,
        0.29544817],
       [0.26729789, 0.40046573, 0.27411645, ..., 0.29850208, 0.29850208,
        0.30002654]])

In [53]:
# Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings_norm, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.478306,0.303464,0.297756,0.371621,0.366645,0.493708,0.420603,0.396858,0.312599,0.346116,0.390644,0.311929,0.313692,0.358193,0.415772,0.346985,0.413841,0.502494,0.520609,0.400155,0.508667,0.312087,0.301166,0.374895,0.333682,0.337258,0.405927,0.409131,0.333381,0.357013,0.362003,0.387883,0.410519,0.354618,0.326673,0.306981,0.350162,0.395982,0.377960,0.400350,0.348058,0.530454,0.469986,0.320913,0.599347,0.384001,0.346985,0.331153,0.329110,0.350309,0.363469,0.364497,0.294719,0.354471,0.297564,0.400824,0.550495,0.452840,0.354289,0.316543,0.323151,0.450239,0.476900,0.508838,0.323265,0.470312,0.359330,0.658183,0.365224,0.340021,0.342263,0.379840,0.347225,0.258025,0.351543,0.348838,0.350102,0.379231,0.361945,0.342424,0.356250,0.478607,0.359200,0.455141,0.302257,0.390131,0.295650,0.318556,0.325310,0.318684,0.572235,0.297885,0.451719,0.392979,0.376377,0.413965,0.334940,0.344393,0.355834,0.390022,0.311910,0.391698,0.458105,0.430275,0.384005,0.320745,0.359192,0.331664,0.406459,0.324609,0.416402,0.402286,0.330175,0.318510,0.405506,0.385797,0.4

## 5.3 CF Recommendation Model (SVD)

In [57]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recRating'})


        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['movieId'].isin(items_to_ignore)] \
                               .sort_values('recRating', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'movieId', 
                                                          right_on = 'movieId')[['recRating', 'movieId', 'title']]

        return recommendations_df

In [58]:
cf_recommender_model = CFRecommender(cf_preds_df,ratings_with_titles_df)

### 5.3.1 Evaluation Result (CF-SVD)

In [59]:
# Instantiate ModelEvaluator with the class definition
model_evaluator_corrected = ModelEvaluator()

# Evaluate the model
global_metrics_corrected, detailed_results_df_corrected = model_evaluator_corrected.evaluate_model(cf_recommender_model)

global_metrics_corrected

{'modelName': 'Collaborative Filtering',
 'recall@5': 0.6919224475628502,
 'recall@10': 0.8083502752020628}

## 5.4 Grid Search (find optimal latent factor K)

In [ ]:
num_factor = np.linspace(1, 50, 50, dtype='int64')

global_recall_5 = []
global_recall_10 = []

for n in num_factor:
    print("Number of factor is: ", n)
    U, sigma, Vt = svds(users_items_pivot_sparse_matrix, k = n)
    sigma = np.diag(sigma)
    
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
    all_user_predicted_ratings_norm = (all_user_predicted_ratings - all_user_predicted_ratings.min()) / \
                                    (all_user_predicted_ratings.max() - all_user_predicted_ratings.min())
    
    cf_preds_df = pd.DataFrame(all_user_predicted_ratings_norm, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
    
    # build model
    cf_recommender_model = CFRecommender(cf_preds_df, movies_df)
    cf_global_metrics, _ = model_evaluator_corrected.evaluate_model(cf_recommender_model)
    
    global_recall_5.append(cf_global_metrics['recall@5'])
    global_recall_10.append(cf_global_metrics['recall@10'])

In [ ]:
# compute the average of Recall@5 & Recall@10
average_metric = 0.5 * (np.array(global_recall_5) + np.array(global_recall_10))

plt.figure(figsize=(12, 6))
plt.plot(num_factor, global_recall_5, label = 'Global Recall@5')
plt.plot(num_factor, global_recall_10, label = 'Global Recall@10')
plt.plot(num_factor, average_metric, label = 'Average performance')

plt.plot(23, 0.365, 'o', ms=5 * 2, mec='red', mfc='none', mew=2)
plt.annotate('Best average', xy=(23, 0.368), xytext=(26, 0.38), fontsize=14,
            arrowprops=dict(facecolor='red', shrink=0.04))

plt.title('CF Performance v.s. Number of factors')
plt.xlabel('Number of factors (k) in SVD')
plt.ylabel('Metrics')
plt.grid()
plt.legend(loc='best')
plt.show()

In [ ]:
index = average_metric.argmax()
k_optimal = num_factor[index]

print("The best number of factor k = ", k_optimal)
print("The corresponding Recall@5 = ", global_recall_5[index])
print("The corresponding Recall@10 = ", global_recall_10[index])

In [ ]:
np.max(global_recall_5), np.max(global_recall_10)

In [ ]:
U, sigma, Vt = svds(users_items_pivot_sparse_matrix, k = k_optimal)
sigma = np.diag(sigma)

all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings_norm = (all_user_predicted_ratings - all_user_predicted_ratings.min()) / \
                                (all_user_predicted_ratings.max() - all_user_predicted_ratings.min())

cf_preds_df = pd.DataFrame(all_user_predicted_ratings_norm, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()

# build model
cf_recommender_model = CFRecommender(cf_preds_df, movies_df)

print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)

print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

In [ ]:
cf_recommender_model_updated = CFRecommender(cf_preds_df, movies_df)

# Try generating recommendations again for the sample user with corrected information
recommendations = cf_recommender_model_updated.recommend_items(1, verbose=True)

recommendations

# Hybrid Recommender 

In [ ]:
class HybridRecommender:
    
    MODEL_NAME = 'Hybrid'
    
    def __init__(self, cb_rec_model, cf_rec_model, items_df, cb_ensemble_weight=1.0, cf_ensemble_weight=1.0):
        self.cb_rec_model = cb_rec_model
        self.cf_rec_model = cf_rec_model
        self.cb_ensemble_weight = cb_ensemble_weight
        self.cf_ensemble_weight = cf_ensemble_weight
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        #Getting the top-1000 Content-based filtering recommendations
        cb_recs_df = self.cb_rec_model.recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose,
                                                           topn=1000).rename(columns={'recRating': 'recRatingCB'})
        
        #Getting the top-1000 Collaborative filtering recommendations
        cf_recs_df = self.cf_rec_model.recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose, 
                                                           topn=1000).rename(columns={'recRating': 'recRatingCF'})
        
        #Combining the results by movieId
        recs_df = cb_recs_df.merge(cf_recs_df,
                                   how = 'outer', 
                                   left_on = 'movieId', 
                                   right_on = 'movieId').fillna(0.0)
        
        #Computing a hybrid recommendation score based on CF and CB scores
        recs_df['recRatingHybrid'] = (recs_df['recRatingCB'] * self.cb_ensemble_weight) \
                                     + (recs_df['recRatingCF'] * self.cf_ensemble_weight)
        
        #Sorting recommendations by hybrid score
        recommendations_df = recs_df.sort_values('recRatingHybrid', ascending=False).head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'movieId', 
                                                          right_on = 'movieId')[['recRatingHybrid', 'movieId', 'title']]


        return recommendations_df

In [ ]:
i = 0.01
while i < 1:
    hybrid_recommender_model = HybridRecommender(content_based_recommender_model, cf_recommender_model_updated, movies_df,
                                             cb_ensemble_weight=i, cf_ensemble_weight=1-i)
    model_evaluator_hybrib= ModelEvaluator()
    print('Evaluating Hybrid model...')
    hybrid_global_metrics, hybrid_detailed_results_df = model_evaluator_hybrib.evaluate_model(hybrid_recommender_model)
    print('\nGlobal metrics:\n%s' % hybrid_global_metrics)
    i += 0.01

In [ ]:
hybrid_recommender_model = HybridRecommender(content_based_recommender_model, cf_recommender_model_updated, movies_df,
                                             cb_ensemble_weight=0.43, cf_ensemble_weight=0.57)

In [ ]:
model_evaluator_hybrib= ModelEvaluator()

In [ ]:
print('Evaluating Hybrid model...')
hybrid_global_metrics, hybrid_detailed_results_df = model_evaluator_hybrib.evaluate_model(hybrid_recommender_model)
print('\nGlobal metrics:\n%s' % hybrid_global_metrics)
hybrid_detailed_results_df.head(10)

# Comparing the methods

In [ ]:
global_metrics_df = pd.DataFrame([cb_global_metrics, pop_global_metrics, global_metrics_corrected, hybrid_global_metrics]) \
                        .set_index('modelName')
global_metrics_df

In [ ]:
%matplotlib inline
ax = global_metrics_df.transpose().plot(kind='bar', figsize=(15,8))
for p in ax.patches:
    ax.annotate("%.3f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

In [ ]:
def inspect_users(person_id, test_set=True):
    if test_set:
        ratings_df = test_indexed_df
    else:
        ratings_df = train_indexed_df
    return ratings_df.loc[person_id].merge(movies_df, how = 'left', 
                                                      left_on = 'movieId', 
                                                      right_on = 'movieId') \
                          .sort_values('rating', ascending = False)[['rating','movieId','title']]

In [ ]:
inspect_users(1, test_set=False).head(10)

In [ ]:
hybrid_recommender_model.recommend_items(1, topn=10, verbose=True)